In [102]:
import os
import re
import numpy as np
import pandas as pd

In [103]:
test_data_files = pd.read_csv (os.path.join ("dataset_test", "file_information.csv"))
test_data_files.head ()

,File,Task,Category
0,g0pA_taska.txt,a,non
1,g0pA_taskb.txt,b,cut
2,g0pA_taskc.txt,c,light
3,g0pA_taskd.txt,d,heavy
4,g0pA_taske.txt,e,non


In [104]:
files = test_data_files['File']
files.head ()

0    g0pA_taska.txt
1    g0pA_taskb.txt
2    g0pA_taskc.txt
3    g0pA_taskd.txt
4    g0pA_taske.txt
Name: File, dtype: object

In [106]:
files_text = []
for file in files :
    files_text.append (open (os.path.join ("dataset_test", file), 'r', encoding = "cp437").read ())

In [107]:
print (len (files_text))

100


In [108]:
files_text[0]

'Inheritance is a basic concept of Object-Oriented Programming where\nthe basic idea is to create new classes that add extra detail to\nexisting classes. This is done by allowing the new classes to reuse\nthe methods and variables of the existing classes and new methods and\nclasses are added to specialise the new class. Inheritance models the\nΓÇ£is-kind-ofΓÇ¥ relationship between entities (or objects), for example,\npostgraduates and undergraduates are both kinds of student. This kind\nof relationship can be visualised as a tree structure, where ΓÇÿstudentΓÇÖ\nwould be the more general root node and both ΓÇÿpostgraduateΓÇÖ and\nΓÇÿundergraduateΓÇÖ would be more specialised extensions of the ΓÇÿstudentΓÇÖ\nnode (or the child nodes). In this relationship ΓÇÿstudentΓÇÖ would be\nknown as the superclass or parent class whereas, ΓÇÿpostgraduateΓÇÖ would\nbe known as the subclass or child class because the ΓÇÿpostgraduateΓÇÖ\nclass extends the ΓÇÿstudentΓÇÖ class.\n\nInheritance can occur 

In [109]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer

nltk.download ('punkt')
nltk.download ('stopwords')
lancaster = LancasterStemmer()

def pre_process (_sentences) :
    en_stops = set (stopwords.words ('english')) # Remove Stopwords
    processed_sentences = []

    for sentence in _sentences :
        sentence = re.sub (r'[^\w\s]','', sentence) # Remove special characters
        lower_sen = sentence.lower () # Convert the sentence to Lowercase
        token_sen = word_tokenize (lower_sen) # Tokenize the sentence

        new_sentence = []
        for word in token_sen:
            if word not in en_stops and word.isnumeric() == False : # Remove words that contain only digits
                word = lancaster.stem (word) # Perform Stemming
                new_sentence.append (word)

        if len (new_sentence) > 3 :
            processed_sentences.append (new_sentence)
    return processed_sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shraeyas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shraeyas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
documents = []
for i, file_text in enumerate (files_text) :
    sentences = file_text.split (".")
    documents.append (pre_process (sentences))

In [111]:
flat_documents = []
for document in documents :
    flat_documents.append ([item for sublist in document for item in sublist])

In [ ]:
flat_documents[50]

In [113]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load (os.path.join ("models", "doc2vec_20news.bin"))

In [114]:
vectors = []
for flat_document in flat_documents :
    v = model.infer_vector (flat_document)
    vectors.append (v)

In [ ]:
vectors[0]

In [ ]:
vectors[50]

In [138]:
from scipy import spatial
values = []
for i in range (len (vectors)) :
    for j in range (i + 1, len (vectors)) :
        result = 1 - spatial.distance.cosine (vectors[i], vectors[j])
        values.append ([result, i + 1, j + 1])

In [139]:
values.sort ()

In [140]:
values[0 : 5]

[[-0.0973544493317604, 44, 51],
 [-0.07829519361257553, 69, 71],
 [-0.07706417143344879, 21, 69],
 [-0.058547187596559525, 69, 96],
 [-0.05392792075872421, 44, 56]]

In [68]:
values[0 : 5]

[[-0.0973544493317604, 44, 51],
 [-0.07829519361257553, 69, 71],
 [-0.07706417143344879, 21, 69],
 [-0.058547187596559525, 69, 96],
 [-0.05392792075872421, 44, 56]]

### Sentence wise similarity
##### We'll calculate sentence wise similarity documents

```
The following function takes 2 documents as input and returns cosine_distance and the start and end index in both the documents for the particular pair
```

In [95]:
def get_similarity (doc_0, doc_1) :
    sentences_dict = {}
    
    sentences_0 = []
    sentences_0 = doc_0.split (".")
    sentences_0_processed = []
    sentences_0_processed.append (pre_process (sentences_0))
    vectors_0 = []
    for i, sentence in enumerate (sentences_0_processed[0]) :
        v = model.infer_vector (sentence)
        for match in re.finditer (sentences_0[i], doc_0) :
            vectors_0.append ([v, match.start (), match.end ()]) 
    
    sentences_1 = []
    sentences_1 = doc_1.split (".")
    sentences_1_processed = []
    sentences_1_processed.append (pre_process (sentences_1))
    vectors_1 = []
    for i, sentence in enumerate (sentences_1_processed[0]) :
        v = model.infer_vector (sentence)
        for match in re.finditer (sentences_1[i], doc_1) :
            vectors_1.append ([v, match.start (), match.end ()])
    
    values = []
    for i in range (len (vectors_0)) :
        for j in range (len (vectors_1)) :
            result = 1 - spatial.distance.cosine (vectors_0[i][0], vectors_1[j][0])
            values.append ([result, [vectors_0[i][1], vectors_0[i][2]], [vectors_1[j][1], vectors_1[j][2]]])
    
    return values

In [97]:
vals = get_similarity (files_text[43], files_text[50])

In [98]:
vals.sort (reverse = True)

In [101]:
vals[0 : 5]

[[0.7193825840950012, [713, 788], [963, 1049]],
 [0.6758325099945068, [713, 788], [1169, 1307]],
 [0.6283330917358398, [713, 788], [845, 962]],
 [0.5072965621948242, [232, 296], [963, 1049]],
 [0.5023082494735718, [713, 788], [381, 527]]]